<a href="https://colab.research.google.com/github/karaage0703/karaage-ai-book/blob/master/ch03/03_karaage_ai_book_generate_text_markov_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# マルコフ連鎖を活用した文章生成

## 教師データのダウンロード

In [ ]:
!wget https://github.com/aozorabunko/aozorabunko/raw/master/cards/000096/files/2093_ruby_28087.zip
!unzip 2093_ruby_28087.zip

--2020-05-06 00:20:57--  https://github.com/aozorabunko/aozorabunko/raw/master/cards/000096/files/2093_ruby_28087.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/000096/files/2093_ruby_28087.zip [following]
--2020-05-06 00:20:57--  https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/000096/files/2093_ruby_28087.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 421747 (412K) [application/zip]
Saving to: ‘2093_ruby_28087.zip’

2093_ruby_28087.zip 100%[===================>] 411.86K  --.-KB/s    in 0.06s   

2020-05-06 00:20:57 (7.00 MB/s) - ‘

ファイルを読み込みます

In [ ]:
text_list = []
with open('dogura_magura.txt', encoding='shift_jis') as f:
  text_list = f.readlines()

In [ ]:
text_list[0:10]

['ドグラ・マグラ\n',
 '夢野久作\n',
 '\n',
 '-------------------------------------------------------\n',
 '【テキスト中に現れる記号について】\n',
 '\n',
 '《》：ルビ\n',
 '（例）蜜蜂《みつばち》\n',
 '\n',
 '｜：ルビの付く文字列の始まりを特定する記号\n']

## データの前処理


形態素解析ライブラリの「janome」をインストール

In [ ]:
!pip install janome

     |████████████████████████████████| 21.5MB 1.4MB/s 


不要な文字の削除

In [ ]:
import re
def normalize_text(text):
    text = re.sub(r'》', '', text)
    text = re.sub(r'※', '', text)
    text = re.sub(r'《', '', text)
    text = re.sub(r'［', '', text)
    text = re.sub(r'＃', '', text)
    text = re.sub(r'-', '', text)
    text = re.sub(r'｜', '', text)
    text = re.sub(r'］', '', text)
    text = re.sub(r'［', '', text)
    text = re.sub(r'【','', text)
    text = re.sub(r'】','', text)
    text = text.strip()
    return text

In [ ]:
new_text_list = []
for text in text_list:
    text = normalize_text(text)
    new_text_list.append(text)

text_list = new_text_list

In [ ]:
text_list[0:10]

['ドグラ・マグラ',
 '夢野久作',
 '',
 '',
 'テキスト中に現れる記号について',
 '',
 '：ルビ',
 '（例）蜜蜂みつばち',
 '',
 '：ルビの付く文字列の始まりを特定する記号']

分かち書き

In [ ]:
from janome.tokenizer import Tokenizer

In [ ]:
def wakachigaki(text_list):
  t = Tokenizer()
  words = []
  for text in text_list:
    tokens = t.tokenize(text)
    for token in tokens:
      pos = token.part_of_speech.split(',')[0]
      words.append(token.surface)

  text = ' '.join(words)
  return text

In [ ]:
word_list = [w for w in wakachigaki(text_list).split()]

In [ ]:
word_list[0:9]

['ドグラ・マグラ', '夢野', '久作', 'テキスト', '中', 'に', '現れる', '記号', 'について', '：']

## マルコフ連鎖モデルの学習

### 簡単な例で確認

In [ ]:
test_text = ['私はからあげが好きだ。君はからあげを食べる。私はおやつが好きだ。']
test_text = wakachigaki(test_text)
test_text = test_text.replace('から あげ', 'からあげ')
test_text = test_text.replace('お やつ', 'おやつ')

test_word_list = [w for w in test_text.split()]
test_word_list

['私',
 'は',
 'からあげ',
 'が',
 '好き',
 'だ',
 '。',
 '君',
 'は',
 'からあげ',
 'を',
 '食べる',
 '。',
 '私',
 'は',
 'おやつ',
 'が',
 '好き',
 'だ',
 '。']

一階のマルコフ連鎖モデル

In [ ]:
def make_markov_model_1(word_list):
  markov = {}
  w1 = ''
  for word in word_list:
    if w1:
      if w1 not in markov:
        markov[w1] = []
      markov[w1].append(word)
    w1 = word
  return markov  

In [ ]:
markov_model_test_1 = make_markov_model_1(test_word_list)

学習したモデルの確認

In [ ]:
def check_model(model, check_numb=10):
  count = 0
  for key in model.keys():
    if count >= 0:
      print('key:', key)
      print('value:', model[key])
      print('------------------------')
    count += 1
    if count > check_numb:
      break

In [ ]:
check_model(markov_model_test_1, check_numb=20)

key: 私
value: ['は', 'は']
------------------------
key: は
value: ['からあげ', 'からあげ', 'おやつ']
------------------------
key: からあげ
value: ['が', 'を']
------------------------
key: が
value: ['好き', '好き']
------------------------
key: 好き
value: ['だ', 'だ']
------------------------
key: だ
value: ['。', '。']
------------------------
key: 。
value: ['君', '私']
------------------------
key: 君
value: ['は']
------------------------
key: を
value: ['食べる']
------------------------
key: 食べる
value: ['。']
------------------------
key: おやつ
value: ['が']
------------------------


In [ ]:
test_text = ['私はからあげが好きだ。君はからあげを食べる。私はおやつが好きだ。空が青い。']
test_text = wakachigaki(test_text)
test_text = test_text.replace('から あげ', 'からあげ')
test_text = test_text.replace('お やつ', 'おやつ')

test_word_list = [w for w in test_text.split()]
test_word_list

['私',
 'は',
 'からあげ',
 'が',
 '好き',
 'だ',
 '。',
 '君',
 'は',
 'からあげ',
 'を',
 '食べる',
 '。',
 '私',
 'は',
 'おやつ',
 'が',
 '好き',
 'だ',
 '。',
 '空',
 'が',
 '青い',
 '。']

In [ ]:
markov_model_test_1 = make_markov_model_1(test_word_list)

In [ ]:
check_model(markov_model_test_1, check_numb=20)

key: 私
value: ['は', 'は']
------------------------
key: は
value: ['からあげ', 'からあげ', 'おやつ']
------------------------
key: からあげ
value: ['が', 'を']
------------------------
key: が
value: ['好き', '好き', '青い']
------------------------
key: 好き
value: ['だ', 'だ']
------------------------
key: だ
value: ['。', '。']
------------------------
key: 。
value: ['君', '私', '空']
------------------------
key: 君
value: ['は']
------------------------
key: を
value: ['食べる']
------------------------
key: 食べる
value: ['。']
------------------------
key: おやつ
value: ['が']
------------------------
key: 空
value: ['が']
------------------------
key: 青い
value: ['。']
------------------------


In [ ]:
def make_markov_model_2(text_list):
  markov = {}
  w1 = ''
  w2 = ''
  for word in text_list:
    if w1 and w2:
      if (w1, w2) not in markov:
        markov[(w1, w2)] = []
      markov[(w1, w2)].append(word)
    w1, w2 = w2, word
  return markov  

In [ ]:
markov_model_test_2 = make_markov_model_2(test_word_list)

In [ ]:
check_model(markov_model_test_2, check_numb=20)

key: ('私', 'は')
value: ['からあげ', 'おやつ']
------------------------
key: ('は', 'からあげ')
value: ['が', 'を']
------------------------
key: ('からあげ', 'が')
value: ['好き']
------------------------
key: ('が', '好き')
value: ['だ', 'だ']
------------------------
key: ('好き', 'だ')
value: ['。', '。']
------------------------
key: ('だ', '。')
value: ['君', '空']
------------------------
key: ('。', '君')
value: ['は']
------------------------
key: ('君', 'は')
value: ['からあげ']
------------------------
key: ('からあげ', 'を')
value: ['食べる']
------------------------
key: ('を', '食べる')
value: ['。']
------------------------
key: ('食べる', '。')
value: ['私']
------------------------
key: ('。', '私')
value: ['は']
------------------------
key: ('は', 'おやつ')
value: ['が']
------------------------
key: ('おやつ', 'が')
value: ['好き']
------------------------
key: ('。', '空')
value: ['が']
------------------------
key: ('空', 'が')
value: ['青い']
------------------------
key: ('が', '青い')
value: ['。']
------------------------


### ドグラ・マグラを使ってモデルを生成

In [ ]:
markov_model_2 = make_markov_model_2(word_list)

In [ ]:
check_model(markov_model_2, check_numb=20)

('ドグラ・マグラ', '夢野')
['久作']
------------------------
('夢野', '久作')
['テキスト', '全集']
------------------------
('久作', 'テキスト')
['中']
------------------------
('テキスト', '中')
['に']
------------------------
('中', 'に')
['現れる', 'ハッキリ', '、', '、', '走り込み', 'ヨボヨボ', '含ま', 'は', '納まっ', '凝固', 'ゴチャゴチャ', '落ち', '、', '追い込ん', '映っ', '突立', '、', '誘い込ん', '湧き', '落し', '並ん', 'は', '封じ込め', '、', '一つ', '、', 'は', '盛込ま', '、', '記述', '記述', '陳列', '含ま', '、', '潜伏', '引寄', 'タッタ', '含ま', 'は', 'は', '摘発', '、', '…', 'は', '落し', '胚胎', '尊', '。', 'は', 'も', '公', '並ぶ', 'は', '交', 'さまよう', '重なる', '。', '居る', 'トグロ', '立て', '立ち止まっ', '発見', '宿っ', '在る', '呼吸', '、', 'ホッ', '、', '、', '葬り', '立往生', '詰めかけ', 'は', '引っ', 'も', '平等', '往々', '、', '見知っ', 'は', '、', '潜在', '寝', '潜在', '包み込ん', '現われ', '含ま', '咲く', '辛', '刺', '落付', '蝨', 'サッと', '留め', '含ま', 'は', '並べ立て', '据え', 'は', '突込', 'は', 'は', '、', 'は', '居る', '居る', '立ち', 'ベタ', 'も', '、', '現われ', '、', 'は', '隠れ', 'は', '息', '盛ら', '、', '潜ん', '納まっ', '一', 'も', '潜ん', 'も', 'は', '絶世', '無理やり', 'ソッ', '納め', '納め', 'も', '恭しく', 'は', '抛', '挿入'

## 文章の生成

### 2階のマルコフモデルで文章生成
作成したモデルを使って文章を生成する

In [ ]:
import random

def generate_text_2(model, max_sentence):
  count_sentence = 0
  sentence = ''
  w1, w2  = random.choice(list(model.keys()))
  while count_sentence < max_sentence:
    try:
      tmp = random.choice(model[(w1, w2)])
      sentence += tmp
      if(tmp=='。'):
        count_sentence += 1
        sentence += '\n'
      w1, w2 = w2, tmp
    except:
      w1, w2  = random.choice(list(model.keys()))

  return sentence

In [ ]:
print(generate_text_2(markov_model_2, 10))

処で今度は方針を変えて隙間すきまもなく私の家に潜り込ませたものである事が好きなのですが、ツイ今しがた失神しかけた時を見澄まして、私の神経中枢とも思えるので、流石さすがの斎藤博士と、法医学者ハ、仮令自身ニ行ワルル犯罪ノ種類モ亦また、本篇の隅々に聞いておれば直ぐに判る筈ですが……▼あ――ア。
その上に在る。
その間かんに場内の片隅だけ引歪ひきゆがめる事がありまして、次から次に襲われながら耳を傾けたまま、黒い制服制帽の、最後の一時間とを押しなべて皆、そうした大昔のマンマの『無常』といった風に、異常な事件を調査するに止とどまった。
「…………」「………一口に日本と違うて。
ザット一千年前の御城下、鳥飼とりかい村自宅に於て、マンモス、エレファス、ステゴドンなぞいうものが、私と肩を一つ二つしたまま突伏つっぷした。
私は驚きの声が次第に静まっているところを見得る程度にまで高潮しているのであるが、この空前絶後の大群集であるにも似合わない小さな、弱々しい咳嗽せきが出た後の今日と相成りましたので、私が指物屋さしものやにで黄色くなった右手の取付とっつきの部屋で見ましても構わないとすれば、眼の前に来てみたいと考えられるのであるが、その祖父というのが習慣になって座り直した。
そうして胎児の潜在意識の内容と、僕はカッと見開いて、お二人の性質を有しおるものに相違ないという事が出来る。
同時に頭の小男で、その前に、寸分違たがわず正確に繰り返して云って聞かせているのでした。
ただの一つ一つに重ねた心理状態に陥って行ける訳さ。
殊ことにはならぬ内容が……ところで今日の正午のお太鼓がドレ位つき難にくいという西洋紙のようになったところを云うのかと思う。



### 2階から5階のマルコフ連鎖に変更

In [ ]:
def make_markov_model_5(word_list):
  markov = {}
  w1 = ''
  w2 = ''
  w3 = ''
  w4 = ''
  w5 = ''
  for word in word_list:
    if w1 and w2 and w3 and w4 and w5:
      if (w1, w2, w3, w4, w5) not in markov:
        markov[(w1, w2, w3, w4, w5)] = []
      markov[(w1, w2, w3, w4, w5)].append(word)
    w1, w2, w3, w4, w5 = w2, w3, w4, w5, word
  return markov

In [ ]:
markov_model_5 = make_markov_model_5(word_list)

In [ ]:
check_model(markov_model_5, 20)

('ドグラ・マグラ', '夢野', '久作', 'テキスト', '中')
['に']
------------------------
('夢野', '久作', 'テキスト', '中', 'に')
['現れる']
------------------------
('久作', 'テキスト', '中', 'に', '現れる')
['記号']
------------------------
('テキスト', '中', 'に', '現れる', '記号')
['について']
------------------------
('中', 'に', '現れる', '記号', 'について')
['：']
------------------------
('に', '現れる', '記号', 'について', '：')
['ルビ']
------------------------
('現れる', '記号', 'について', '：', 'ルビ')
['（']
------------------------
('記号', 'について', '：', 'ルビ', '（')
['例']
------------------------
('について', '：', 'ルビ', '（', '例')
['）']
------------------------
('：', 'ルビ', '（', '例', '）')
['蜜蜂']
------------------------
('ルビ', '（', '例', '）', '蜜蜂')
['みつば']
------------------------
('（', '例', '）', '蜜蜂', 'みつば')
['ち']
------------------------
('例', '）', '蜜蜂', 'みつば', 'ち')
['：']
------------------------
('）', '蜜蜂', 'みつば', 'ち', '：')
['ルビ']
------------------------
('蜜蜂', 'みつば', 'ち', '：', 'ルビ')
['の']
------------------------
('みつば', 'ち', '：', 'ルビ', 'の')
['付く']
------------------------
(

In [ ]:
import random

def generate_text_5(model, max_sentence):
  count_sentence = 0
  sentence = ''
  w1, w2, w3, w4, w5  = random.choice(list(model.keys()))
  while count_sentence < max_sentence:
    try:
      tmp = random.choice(model[(w1, w2, w3, w4, w5)])
      sentence += tmp
      if(tmp=='。'):
        count_sentence += 1
        sentence += '\n'
      w1, w2, w3, w4, w5 = w2, w3, w4, w5, tmp
    except:
      w1, w2, w3, w4, w5  = random.choice(list(model.keys()))

  return sentence

In [ ]:
print(generate_text_5(markov_model_5, 10))

なのか知らん。
それとも二人の博士が、チットモ怖くなくなった許ばかりでなく、妹にも久しく不品行ふしだらな事が御座いません事が、亡骸なきがらをお調べ下さいましてから、お判りになりましたとの事で、総長室の隣室で聞いていた事務員連は皆、同教授の発狂を疑いつつ顔を見合わせつつ震え上ったという。
鼾声かんせい雷らいの如く「鼾声雷の如く」は５段階大きな文字気味悪い屍体「気味悪い屍体」は５段階大きな文字気味悪い屍体「気味悪い屍体」は５段階大きな文字同時に狂人の解放治療場に於ける呉一郎の女性絞殺行為後の夢中遊行症は殆ど右と同様のものなるべけれども、更に、ここに変態性慾的内容を有する夢中遊行を添加したる形跡の明らかなるものあるは特に珍重頑味すべきところなり。
即ち呉一郎は、自己の血統に伝われる、独特固有の、変態性慾的「心理遺伝」の参考材料としても、その価値は形容の出来ない程に素晴らしいものがある。
しかも、そいつに釣り込まれて、無我夢中に読み続けていたので……キチガイ地獄外道祭文「キチガイ地獄外道祭文」は５段階大きな文字ここから９字下げ――一名、狂人の暗黒時代――ここで字下げ終わり而しかして人間の肉体、及び精神と、細胞の霊能との関係が、斯様に明白となった以上「夢」なるものの本質に関する説明も亦また、極めて順調、正確に、精細をきわめつつ移りかわって行く。
この点が、勝手気儘な自己陶酔に陥って行ける訳さ。
気持ちの純な、頭のいい人間の変態心理は、ナカナカ見分けが付きにくいんだよ。
……ところでその無言の所作が、開幕の皮切りに、大衆に投げかけた疑問というのは『私は誰の児こか』という質問であった。
……その一箇月前の十月二十日だよ。
いいかい。



# 参考リンク
- https://omedstu.jimdofree.com/2018/05/06/%E3%83%9E%E3%83%AB%E3%82%B3%E3%83%95%E9%80%A3%E9%8E%96%E3%81%AB%E3%82%88%E3%82%8B%E6%96%87%E6%9B%B8%E7%94%9F%E6%88%90/
- https://qiita.com/k-jimon/items/f02fae75e853a9c02127